In [47]:
import pandas as pd
import re

In [2]:
client_info_df = pd.read_csv(r'E:\wd_work\unti_cheat\D1_B_0201_0613_0615_B.csv')

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
client_info_df.columns

Index(['ACTION_CONTROL', 'ACTION_GROUP', 'ACTION_PAGE', 'ACTION_TEXT',
       'ACTION_VALUE', 'CLIENT_TIME', 'CREATE_DATE', '中文匹配', 'D3', 'D10', 'F6',
       'GETTEL(A.MOBILE)', 'ID', 'MACHINE_UUID', 'MAX_MAXOVERDUEDAYS', '记录数',
       'PUTOUTDATE_TIMES', 'T3%', 'T30#', 'T3#', 'T10#', '[T3->T10]%', 'T10% ',
       'T20#', 'TEL', 'USER_ID', '日(APPLY_DATE)', '日(MIN_PUTOUTDATE)'],
      dtype='object')

In [5]:
client_info_df.ACTION_CONTROL.value_counts()

my_home_address        815662
my_company_name        524858
my_contact_phone       376697
my_contact             319647
my_work_address        137682
city                    30662
my_phone                12943
my_contact_relation      9442
Name: ACTION_CONTROL, dtype: int64

In [7]:
company_name_df = client_info_df[client_info_df.ACTION_CONTROL==u'my_company_name']

In [62]:
company_name_df1 = company_name_df[['USER_ID','ID','ACTION_VALUE']].sort_values(by=['USER_ID','ID'])
company_name_df1.ACTION_VALUE = company_name_df1.ACTION_VALUE.apply(lambda x:re.sub(u"[a-z？]",'',str(x)))
company_name_df1.ACTION_VALUE = company_name_df1.ACTION_VALUE.apply(lambda x:re.sub(u"[a-z0-9]",'',str(x)))
company_name_df1.ACTION_VALUE = company_name_df1.ACTION_VALUE.apply(lambda x:re.sub(u"[？ ']",'',str(x)))

In [63]:
company_name_df1['company_name_len'] = company_name_df1.ACTION_VALUE.str.len()

In [64]:
company_name_df1['company_name_len1'] = company_name_df1['company_name_len'].shift(-1)
company_name_df1['company_name_len2'] = company_name_df1['company_name_len'].shift(1)

In [65]:
company_name_df1['maxmum_index'] = 0
company_name_df1.ix[(company_name_df1['company_name_len']>company_name_df1['company_name_len1'])&(company_name_df1['company_name_len']>company_name_df1['company_name_len2']),'maxmum_index']=1
def set_last_1(df):
    df.iloc[-1,-1] = 1
    return df
company_name_df1 = company_name_df1.groupby('USER_ID').apply(set_last_1)

In [66]:
company_name_df2 = company_name_df1[company_name_df1.maxmum_index==1]

In [67]:
company_name_df3 = pd.merge(company_name_df2,company_name_df2[['USER_ID','ID']].groupby('USER_ID').count().reset_index().rename(columns={'ID':'ID_NUM'}),on='USER_ID',how='left')

In [68]:
company_name_df4 = company_name_df3[company_name_df3.ID_NUM==1]

In [69]:
company_name_df5 = company_name_df3[company_name_df3.ID_NUM!=1]

In [74]:
import difflib
def get_company_name(df):
    company_name_list = list(df.ACTION_VALUE)
    company_name = [company_name_list[-1],]
    for name in company_name_list[:-1]:
        if name not in company_name_list[-1] and difflib.SequenceMatcher(None,name,company_name_list[-1]).ratio()<0.4:
            company_name.append(name)
    return len(company_name)
company_name_df6 = company_name_df5.groupby('USER_ID').apply(get_company_name).reset_index().rename(columns={0:'company_name_num'})

In [78]:
company_name_df7 = company_name_df4[['USER_ID','ID_NUM']].rename(columns={'ID_NUM':'company_name_num'})

In [80]:
company_name_df8 = pd.concat([company_name_df6,company_name_df7])

In [81]:
company_name_df8['is_companyname_changed'] = 0
company_name_df8.ix[company_name_df8.company_name_num>1,'is_companyname_changed'] =1

In [84]:
is_companyname_changed_df = company_name_df8[['USER_ID','is_companyname_changed']]

In [ ]:
#####################输入公司名称熟练度###########################

In [86]:
%%time
company_name_df = client_info_df[client_info_df.ACTION_CONTROL=='my_company_name']
company_name_df = company_name_df.sort_values(by=['USER_ID','ID'])
company_name_df['company_name_len'] = company_name_df.ACTION_VALUE.str.len()
def cal_pro(df):
    df['company_name_len1'] = df['company_name_len'].shift(-1).fillna(0)
    df['dif'] = df['company_name_len1']-df['company_name_len']
    df1 = df[df.dif>0]
    if len(df1)>0:
        a = sum(df1.dif)/len(df1)
    else:
        a = -1
    return a
input_companyanme_proficiency_df = company_name_df.groupby('USER_ID').apply(cal_pro)

Wall time: 43.4 s


In [ ]:
################公司类别划分#############################

In [87]:
#################是否包含省市############################

In [88]:
%%time
company_name_df = client_info_df[client_info_df.ACTION_CONTROL=='my_company_name']
company_name_df = company_name_df.sort_values(by=['USER_ID','ID']).drop_duplicates('USER_ID',keep='last')[['USER_ID','ACTION_VALUE']]
company_list = list(company_name_df.ACTION_VALUE)
company_list_box = []
for i in range(len(company_list)):
    if ('省' in company_list[i]) |('市' in company_list[i]) |('县' in company_list[i]):
        company_list_box.append(1)
    else:
        company_list_box.append(0)
company_name_df['is_companyname_includecity'] = company_list_box
is_companyname_includecity_df = company_name_df[['USER_ID','is_companyname_includecity']]

Wall time: 1.13 s


In [89]:
###############名称长度######################

In [91]:
%%time
company_name_df = client_info_df[client_info_df.ACTION_CONTROL=='my_company_name']
company_name_df = company_name_df.sort_values(by=['USER_ID','ID']).drop_duplicates('USER_ID',keep='last')[['USER_ID','ACTION_VALUE']]
company_name_df['company_name_len'] = company_name_df.ACTION_VALUE.str.len()
company_name_len_df = company_name_df[['USER_ID','company_name_len']]

Wall time: 788 ms


In [ ]:
#############文字当中是否有括号或其他标点#################

In [93]:
%%time
company_name_df = client_info_df[client_info_df.ACTION_CONTROL=='my_company_name']
ll = []
userlist = []
for USER_ID in set(list(company_name_df.USER_ID)):
    ll.append(''.join([str(company_name) for company_name in list(company_name_df[company_name_df.USER_ID==USER_ID].ACTION_VALUE)]))
    userlist.append(USER_ID)
is_obtain_someting_list = []
for l in ll:
    c = 0
    for a in ['(',')','（','）','【','】','[',']','{','}','-','_']:
        if a in l:
            c += 1
    if c == 0:
        is_obtain_someting_list.append(0)
    else:
        is_obtain_someting_list.append(1)
is_companyname_include_punctuation = pd.DataFrame({'USER_ID':userlist,'is_companyname_include_punctuation':is_obtain_someting_list}) 

Wall time: 24.2 s


In [94]:
#############是否有过复制粘贴#################

In [95]:
%%time
company_name_df = client_info_df[client_info_df.ACTION_CONTROL=='my_company_name']
company_name_df = company_name_df.sort_values(by=['USER_ID','ID']).drop_duplicates('USER_ID',keep='first')[['USER_ID','ACTION_VALUE']]
company_name_df['company_name_len'] = company_name_df.ACTION_VALUE.str.len()
company_name_df['is_company_copypaste'] = 0
company_name_df.ix[company_name_df['company_name_len']>=5,'is_company_copypaste'] = 1
is_companyname_copypaste_df = company_name_df[['USER_ID','is_company_copypaste']]

Wall time: 804 ms


In [96]:
###############总编辑次数######################

In [97]:
%%time
company_name_df = client_info_df[client_info_df.ACTION_CONTROL=='my_company_name']
company_id_cnt = company_name_df[['USER_ID','ID']].groupby('USER_ID').count().reset_index().rename(columns={'ID':'company_id_cnt'})

Wall time: 327 ms


In [98]:
#############首次输入字符长度#######################

In [100]:
%%time
company_name_df = client_info_df[client_info_df.ACTION_CONTROL=='my_company_name']
company_name_df = company_name_df.sort_values(by=['USER_ID','ID']).drop_duplicates('USER_ID',keep='first')[['USER_ID','ACTION_VALUE']]
company_name_df['company_name_len'] = company_name_df.ACTION_VALUE.str.len()
first_companyinput_strlen_df = company_name_df[['USER_ID','company_name_len']].rename(columns={'company_name_len':'first_companyinput_strlen'})

Wall time: 767 ms
